In [12]:
from pydent import AqSession, models, ModelBase
import pprint

In [9]:
#Import data analysis tools...

# Plot things in the notebook, instead of on a separate window.
%matplotlib inline

import numpy as np # imports a fast numerical programming library
import scipy as sp #imports stats functions, amongst other things
import matplotlib as mpl # this actually imports matplotlib
import matplotlib.cm as cm #allows us easy access to colormaps
import matplotlib.pyplot as plt #sets up plotting under plt
import pandas as pd #lets us handle data as dataframes
#sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns #sets up styles and gives us more plotting options

In [20]:
s = AqSession("Nick", "25Me4qvvvv", "http://54.190.2.203")
s.set_timeout(60)

In [21]:
# #Collect data on ops from plans submitted by users with these usernames
# COLLECT_USERS = ["Nick (Real User)"]

# #Collect data on operation types with these names
# COLLECT_OPERATION_TYPES = ["Check Plate", "Purify Gel Slice", "Make Miniprep", "Purify DNA from Reaction", "Upload Sequencing Results"]

# #Specifies data associations to collect for output items.
# #Example:
# # COLLECT_OUTPUT_ITEM_DATA = {
#     #"Operation name": {
#         #"An output name": ["data association name", "another data association name"]
#         #},
#     #"Another operation name": {
#         # "An output name": ["data association name", "another data association name"]
#     #}
# #}
# COLLECT_OUTPUT_ITEM_DATA = {
#     "Check Plate": {
#         "Plate": ["white_colonies", "blue_colonies", "red_colonies"]
#     },
#     "Purify Gel Slice": {
#         "Fragment": ["concentration", "volume", "A260280", "A260230"]
#     },
#     "Make Miniprep": {
#         "Plasmid": ["concentration", "volume", "A260280", "A260230"]
#     },
#     "Purify DNA from Reaction": {
#         "Purified DNA": ["concentration", "volume", "A260280", "A260230"]
#     },
#     "Upload Sequencing Results": {
#         "Plasmid": ["sequence_ok", "delete_requested"]
#     }
# }


In [45]:
# collect_user_ids = [s.User.find_by_name(u).id for u in COLLECT_USERS]

# #fig, ax = plt.subplots(COLLECT_OPERATION_TYPES, 1)

# for i, ot_name in enumerate(COLLECT_OPERATION_TYPES):
#     op_data = {
#         "id" : [],
#         "time" : [],
#         "technician" : [],
#         "status" : []
#     }
    
#     for user_id in collect_user_ids:
#         print(ot_name)
        
#         ot = s.OperationType.where({"name": ot_name, "deployed": True})[0]

#         ops = [op for op in ot.operations if ((op.status == 'done' or op.status == 'error') and op.user_id == user_id)]
#         ops = ops[:20]

#     #     #List used to store data for all operations of this operation type.
#     #     op_data = []
#     #     for op in ops:
#     #         if s.User.find(op.plan.user_id).name in COLLECT_USERS:
#     #             op_data.append({
#     #                 "ID": op.id,
#     #                 "Status": op.status,
#     #                 "Submitted By": s.User.find(op.plan.user_id).name
#     #             })
#     #             if op.jobs:
#     #                 op_data[-1]["Run By"] = s.User.find(op.jobs[-1].user_id).name
#     #             else:
#     #                 op_data[-1]["Run By"] = ""
#     #             if COLLECT_OUTPUT_ITEM_DATA[ot_name]:
#     #                 for output_name, output_data in COLLECT_OUTPUT_ITEM_DATA[ot_name].items():
#     #                     output_item = op.output(output_name).item
#     #                     for data_name in output_data:
#     #                         if output_item and s.DataAssociation.where({"parent_id": output_item.id, "key": data_name}):
#     #                             val = s.DataAssociation.where({"parent_id": output_item.id, "key": data_name})[0].value
#     #                             op_data[-1]["Output ({0}), {1}".format(output_name, data_name)] = val
#     #                         else:
#     #                             op_data[-1]["Output ({0}), {1}".format(output_name, data_name)] = ""
#     #     print(op_data)

#         for op in ops:
#             op_data["id"].append(op.id)
#             if op.jobs:
#                 op_data["time"].append(op.jobs[-1].updated_at)
#                 op_data["technician"].append(op.jobs[-1].user_id)
#             else:
#                 op_data["time"].append(None)
#                 op_data["technician"].append(None)
#             op_data["status"].append(op.status)

#     #df = pd.DataFrame(op_data)
#     #print(df["time"])
#     #df["time"].plot_date(kind='hist', axs=ops_over_time_axs[i])
    
    

Check Plate


AttributeError: 'str' object has no attribute 'astype'